In [3]:
using DataFrames, CSV
using JuMP, Gurobi
using LinearAlgebra, Random, Printf
using Plots

const GRB_ENV = Gurobi.Env()

LoadError: [91mGurobi Error 10009: Failed to obtain a valid license[39m

In [ ]:
Pb1_arcs.csv: A 90×5 matrix reporting, for each arc (column 1), the starting and ending
nodes (columns 2 and 3), the capacity (column 4) and the construction cost (column 5).
– Pb2_transport.csv: A 90×3 matrix reporting the unit transportation cost on each arc (rows)
for each customer (columns).
– Pb2_demand.csv: A 3×2 matrix reporting the demand from each customer (rows) in each
scenario (columns).
– Pb2_customer_OD.csv: A 3×2 matrix reporting, for each customer (rows), the origin (column
1) and the destination (column 2).

In [7]:
arcs = CSV.read("Pb1_arcs.csv", DataFrame)
transport = CSV.read("Pb1_transport.csv", DataFrame)
demand = CSV.read("Pb1_demand.csv", DataFrame)
customers = CSV.read("Pb1_customer_OD.csv", DataFrame)

,x1,x2
,Int64,Int64
1,1,9
2,4,2
3,8,2


In [13]:
size(arcs)

(90, 5)

## 1. Problem setup

We want to solve the following stochastic facility location problem:

$$\begin{align}
\min\quad & \sum_{i=1}^Nc_ix_i+\sum_{s=1}^Sp_s\left(\sum_{i=1}^N\sum_{j=1}^Mt_{ij}y_{ij}^s + \sum_{j=1}^Mq_jz_{j}^s\right)\\
\text{s.t.}\quad & \sum_{i=1}^Ny_{ij}^s+z_j^s\ge d_j^s & \forall j\in [M], s\in [S]\\
& \sum_{j=1}^My_{ij}^s \le C_ix_i&\forall i\in[N], s\in[S]\\
& \mathbf{y,z}\ge 0, \mathbf{x}\in\{0,1\}^N
\end{align}$$

Dual subproblem $s$:
$$\begin{align}
\max\quad &  \sum_{j=1}^M \mu_j d_j^s - \sum_{i=1}^N\lambda_iC_ix_i\\
\text{s.t.}\quad & \mu_j -\lambda_i \le t_{ij} & \forall i\in[N], j\in[M]\\
& \mu_j \le q_j &\forall j\in [M]\\
& \mathbf{\mu, \lambda}\ge 0
\end{align}$$

- If the dual subproblem is unbounded, we obtain an extreme ray $(\mu^*, \lambda^*)$ and add a feasibility cut:
$$\sum_{j=1}^M\mu^*_j d_j^s - \sum_{i=1}^N \lambda^*_i C_i x_i \le 0$$

- If the dual subproblem solves to optimality, we obtain an extreme point $(\mu^*, \lambda^*)$ and add an optimality cut:
$$θ_s \ge \sum_{j=1}^M\mu^*_j d_j^s - \sum_{i=1}^N\lambda^*_i C_i x_i$$

\begin{equation}
    \label{eq:subproblem-stochastic-dual}
    \begin{array}{ll}
        \max & \sum_{k \in \mathcal{K}} d_k^s ({\pi_{O_k, k}^s - \pi_{D_k,k}^s}) + \sum_{(i, j) \in \mathcal{A}} u_{i j} x_{i j} \mu_{ i j}^s \\
        \text { s.t. } & \pi_{i, k}^s - \pi_{j, k}^s + \mu_{i j}^s \leq f_{i j k}, \forall (i, j) \in \mathcal{A}, k \in \mathcal{K} \\
        & \pi_{i k}^s \in \mathbb{R}, \forall i \in \mathcal{N}, k \in \mathcal{K} \\
        & \mu_{i j}^s \leq 0,  \forall(i, j) \in \mathcal{A} \\
    \end{array}
\end{equation}

In [4]:
"Solve problem using multi-cut Benders decomposition"
function solve_benders_multi(data::FacilityLocationData; verbose::Bool=false)
    # define main problem
    MP = Model(() -> Gurobi.Optimizer(GRB_ENV));
    set_optimizer_attributes(MP, "TimeLimit" => 60, "MIPGap" => 1e-4, "OutputFlag" => 0)
    N = size(arcs, 1); M = size(customers, 1); S = size(demand, 2)
    @variable(MP, x[1:N], Bin)
    @variable(MP, θ[1:S] >= 0)
    @objective(MP, Min, sum(arcs[3][i] * x[i] for i=1:N) + sum(1/S * θ[s] for s in 1:S))

    lower_bound_all = []; upper_bound_all = []
    MP_time = []; SP_max_time = []; SP_time = []
    while true
        # solve master problem
        push!(MP_time, @elapsed optimize!(MP))
        lower_bound_new = objective_value(MP)
        push!(lower_bound_all, lower_bound_new)
        x_MP = value.(MP[:x])
        # solve S subproblems
        obj_SP = zeros(S)
        SP_time_all = zeros(S)
        for s = 1:S
            SP_dual = Model(() -> Gurobi.Optimizer(GRB_ENV))
            set_optimizer_attributes(SP_dual, "OutputFlag" => 0)
            @variable(SP_dual, λ[1:N] >= 0);
            @variable(SP_dual, μ[1:M] >= 0);
            @objective(SP_dual, Max,
                       sum(μ[j] * demand[j,s] for j in 1:M) -
                       sum(λ[i] * arcs[4][i] * x_MP[i] for i in 1:N))
            @constraint(SP_dual, [i in 1:N, j in 1:M], μ[j] - λ[i] <= transport[i,j])
            @constraint(SP_dual, [j in 1:M], μ[j] <= data.cost_unmet_demand[j])
            SP_time_all[s] = @elapsed optimize!(SP_dual)
            obj_SP_dual = objective_value(SP_dual)
            λ_val = value.(SP_dual[:λ])
            μ_val = value.(SP_dual[:μ])            
            if termination_status(SP_dual) == MOI.DUAL_INFEASIBLE # feasibility cut
                @constraint(MP, sum(μ_val[j] * demand[j, s] for j in 1:M) -
                            sum(λ_val[i] * arcs[4][i] * x[i] for i in 1:N) <= 0)
                obj_SP[s] = 999999999
            elseif termination_status(SP_dual) == MOI.OPTIMAL
                @constraint(MP, θ[s] >= sum(μ_val[j] * data.demand[j,s] for j in 1:M) -
                            sum(λ_val[i] * data.capacity[i] * x[i] for i in 1:N))
                obj_SP[s] = obj_SP_dual
            end
        end
        push!(SP_max_time, maximum(SP_time_all))
        push!(SP_time, sum(SP_time_all))
        upper_bound_new = sum(data.facility_cost[i] * x_MP[i] for i=1:N) + sum(data.prob[s] * obj_SP[s] for s in 1:S)
        push!(upper_bound_all, upper_bound_new)
        verbose && @printf("Sol: %.2f - Bound: %.2f\n", upper_bound_all[end], lower_bound_all[end])
        if sum(MP_time) + sum(SP_time) >= TIME_LIMIT ||
            (upper_bound_new-lower_bound_new)/lower_bound_new < OPTIMALITY_GAP
            break
        end
    end
    return upper_bound_all, lower_bound_all, MP_time, SP_time, SP_max_time
end

LoadError: [91mUndefVarError: FacilityLocationData not defined[39m